In [31]:
#TODO nao estou tratanto o caso do OR
#TODO nao nao trado caso o usario queira colocar um intervalor tipo assim 30<=var<=40

In [32]:
import json
from pyparsing import Word, alphas, nums, oneOf, infixNotation, opAssoc, ParserElement
from sympy import sympify
from sympy.logic.boolalg import to_dnf
import re

In [33]:
def get_desired_scenario(file_path = 'analyzed_scenario_process.json'):
    # Ler o arquivo JSON
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Pegar o valor da chave 'desired'
    desired_scenario = data.get('desired')
    # desired_scenario_name = next(iter(desired_scenario))
    # Exibir o valor

    # print("Desired scenario: ", desired_scenario)
    
    return desired_scenario

# get_desired_scenario()

In [34]:
def load_variable_ranges(file_path="variables_config.json"):
    with open(file_path, 'r') as file:
        json_data = file.read()
    variable_ranges = json.loads(json_data)
    
    # print("Variable ranges: ", {var['name']: var for var in variable_ranges})

    return {var['name']: var for var in variable_ranges}

# load_variable_ranges()

In [35]:
def get_shared_scenarios(file_path = 'shared_base.json'):
    # Ler o arquivo JSON
    with open(file_path, 'r') as file:
        shared_scenarios = json.load(file)
    return shared_scenarios

In [36]:
def get_Given(scenario):    
    scenario_name = next(iter(scenario))
    given_exp=scenario[scenario_name]["Given"]
    # print("Given: ",given_exp)
    return given_exp

# print(get_Given(get_desired_scenario()))

In [37]:
def get_When(scenario):    
    scenario_name = next(iter(scenario))
    when_exp=scenario[scenario_name]["When"]
    # print("When: ", when_exp)
    return when_exp

# print(get_When(get_desired_scenario()))

In [38]:
def get_Then(scenario):    
    scenario_name = next(iter(scenario))
    then_exp=scenario[scenario_name]["Then"]
    # print("Then: ", then_exp)
    return then_exp

# print(get_Then(get_desired_scenario()))

In [39]:
def get_precondition(scenario):

    given_exp=get_Given(scenario)
    when_exp=get_When(scenario)

    precond = "{} AND {}".format(given_exp, when_exp)
    # print(f"Precondition: {precond}")
    return precond

# print(get_precondition(get_desired_scenario()))

In [40]:
def get_postcondition(scenario):

    postcond =  get_Then(scenario)
    # print(f"Postcondition: {postcond}")
    return postcond

# print(get_postcondition(get_desired_scenario()))

In [41]:
def logical_expression_settings():
    # Configurar pyparsing para ignorar espaços em branco
    ParserElement.enablePackrat()

    # Definir elementos básicos da expressão
    identifier = Word(alphas, alphas + nums + "_")
    number = Word(nums)

    # Definir operadores de comparação
    comparison_operator = oneOf("<= >= < > == !=")

    # Definir expressão básica
    operand = identifier | number

    # Definir expressão lógica
    logical_expr = infixNotation(operand,
        [
            (comparison_operator, 2, opAssoc.LEFT),
            ("AND", 2, opAssoc.LEFT),
            ("OR", 2, opAssoc.LEFT),
        ])

    return logical_expr


def extract_leaves(parsed_expr):
    """Extrai folhas de uma expressão lógica analisada."""
    
    if isinstance(parsed_expr, str):
        return []
    elif len(parsed_expr) == 3 and parsed_expr[1] in ["<=", ">=", "<", ">", "==", "!="]:
        return [" ".join(parsed_expr)]
    else:
        leaves = []
        for sub_expr in parsed_expr:
            leaves.extend(extract_leaves(sub_expr))
        return leaves

# Exemplo de uso
# expression_scenario = "(((a <= 3 AND (b > 10 OR c == 10)) OR ((d != 5 AND e >= 20) OR f < 2)) AND ((g <= 7 OR h > 8) AND (i == 9 OR j != 3)))"
# expression_scenario = "(a <= 3 AND b > 10)"

# expression_scenario = get_Given(get_desired_scenario())

# parsed_expression = logical_expression_settings().parseString(expression_scenario, parseAll=True)

# print("Extracted Leafs:", extract_leaves(parsed_expression.asList()))


In [42]:
def list_to_indexed_dict(lst):
    """
    Transforms a list of strings into a dictionary with keys 'x1', 'x2', ..., 'xn'.
    
    Parameters:
    lst (list): List of strings.

    Returns:
    dict: Dictionary with keys 'x1', 'x2', ..., 'xn' and values from the original list.
    """
    return {f'x{i+1}': value for i, value in enumerate(lst)}

# Example usage
# lst = ['a <= 3', 'b > 10']
# map_dict = list_to_indexed_dict(extract_leaves(parsed_expression.asList()))
# print("Indexed Dict:", map_dict)


In [43]:
def replace_values_with_keys(string, dictionary):
    """
    Replaces occurrences of the dictionary values with the keys in the string.

    Parameters:
    string (str): The string where the replacements will be made.
    dictionary (dict): The dictionary with keys and values to be replaced.

    Returns:
    str: The string with values replaced by keys.
    """
    for key, value in dictionary.items():
        string = string.replace(value, key)
    return string

# indexed_scenario = replace_values_with_keys(expression_scenario, map_dict)
# print(indexed_scenario)


In [44]:
def replace_logical_operators(string, scenario_operators=False):
    """
    Substitui os operadores lógicos 'AND' por '&' e 'OR' por '|', ou vice-versa.

    Parâmetros:
    string (str): A string onde as substituições serão feitas.
    scenario_operators (bool): Se True, substitui '&' por 'AND' e '|' por 'OR'.
                               Se False, substitui 'AND' por '&' e 'OR' por '|'.

    Retorno:
    str: A string com os operadores lógicos substituídos.
    """
    if scenario_operators:
        return string.replace('&', 'AND').replace('|', 'OR')
    else:
        return string.replace('AND', '&').replace('OR', '|')



# indexed_scenario = replace_logical_operators(indexed_scenario)
# print("indexed_scenario:", indexed_scenario)

In [45]:
def _to_dnf(indexed_scenario):
    """
    Converts a logical expression to Disjunctive Normal Form (DNF).

    Parameters:
    indexed_scenario (str): The logical expression in string format with indexed variables.

    Returns:
    str: The logical expression in Disjunctive Normal Form.
    """
    # Convert the string to a symbolic expression
    logical_expression = sympify(indexed_scenario)

    # Convert the symbolic expression to Disjunctive Normal Form
    dnf_expr = to_dnf(logical_expression, simplify=True, force=True)

    return str(dnf_expr)

# Example usage
# dnf_expr = __to_dnf_expression(indexed_scenario)
# print(dnf_expr)

In [46]:
def revert_keys_to_values(string, dictionary):
    """
    Replaces occurrences of the dictionary keys with the values in the string.

    Parameters:
    string (str): The string where the replacements will be made.
    dictionary (dict): The dictionary with keys and values to be replaced.

    Returns:
    str: The string with the keys replaced by the original values.
    """
    for key, value in dictionary.items():
        string = string.replace(key, value)
    
    return string

# Example usage
# final_string = revert_keys_to_values(str(dnf_expr), map_dict)
# print(f"Final string with original values: {final_string}")


In [47]:
def to_dnf_expression(expression_scenario):

    parsed_expression = logical_expression_settings().parseString(expression_scenario, parseAll=True)
    # print("Parsed Expression:", parsed_expression)

    extracted_leaves=extract_leaves(parsed_expression.asList())
    # print("Extracted Leafs:", extracted_leaves)

    map_dict = list_to_indexed_dict(extracted_leaves)
    # print("Indexed Dict:", map_dict)

    indexed_scenario = replace_values_with_keys(expression_scenario, map_dict)
    # print("replaced values with keys:", indexed_scenario)

    indexed_scenario = replace_logical_operators(indexed_scenario)
    # print("indexed_scenario:", indexed_scenario)

    dnf_expr = _to_dnf(indexed_scenario)
    # print(dnf_expr)

    # Example usage
    dnf_expression = revert_keys_to_values(str(dnf_expr), map_dict)
    # print(f"Final string with original values: {dnf_expression}")

    dnf_expression=replace_logical_operators(dnf_expression, scenario_operators=True)
    # print(f"dnf_expression: {dnf_expression}")

    return dnf_expression

In [48]:
#VERSAO 1
##############
# import re
# import json

# def parse_expression(expression):
#     """
#     Parse the expression into its components: variable, operator, and value.
#     """
#     match = re.match(r"(\w+)\s*(>=|<=|>|<|==|!=)\s*(.+)", expression)
#     if match:
#         variable, operator, value = match.groups()
#         return variable, operator, value
#     else:
#         raise ValueError(f"Invalid expression format: {expression}")

# def load_variable_ranges_from_file(file_path):
#     with open(file_path, 'r') as file:
#         json_data = file.read()
#     variable_ranges = json.loads(json_data)
#     return {var['name']: var for var in variable_ranges}

# # def normalize_value(value, var_type):
# #     """
# #     Normalize the value based on its type (integer or float).
# #     """
# #     if var_type == 'integer':
# #         return int(value)
# #     elif var_type == 'float':
# #         return float(value)
# #     else:
# #         raise ValueError(f"Unsupported variable type: {var_type}")


# def get_expression_range(var_name, operator, value, min_val, max_val):
#     """
#     Determine the range of values that satisfy the expression.
#     """
#     value = float(value)
#     if operator == '>=':
#         return (value, max_val)
#     elif operator == '<=':
#         return (min_val, value)
#     elif operator == '>':
#         return (value + 1e-9, max_val)  # Adding a small value to make it exclusive
#     elif operator == '<':
#         return (min_val, value - 1e-9)  # Subtracting a small value to make it exclusive
#     elif operator == '==':
#         return (value, value)
#     elif operator == '!=':
#         return [(min_val, value - 1e-9), (value + 1e-9, max_val)]  # Returns two ranges
#     else:
#         raise ValueError(f"Unsupported operator: {operator}")

# def calculate_jaccard_similarity(expr1, expr2, variable_ranges):
#     """
#     Calculate the Jaccard similarity between two expressions.
#     The expressions should be of the format: "variable operator value".
#     """
#     try:
#         var1, op1, val1 = parse_expression(expr1)
#         var2, op2, val2 = parse_expression(expr2)
#     except ValueError as e:
#         print(e)
#         return 0.0

#     # Ensure the variable names are the same
#     if var1 != var2:
#         return 0.0

#     # Get variable ranges
#     var_info = variable_ranges.get(var1)
#     if not var_info:
#         print(f"Variable {var1} not found in ranges.")
#         return 0.0

#     min_val = var_info['min_value']
#     max_val = var_info['max_value']

#     # Get ranges for each expression
#     range1 = get_expression_range(var1, op1, val1, min_val, max_val)
#     range2 = get_expression_range(var2, op2, val2, min_val, max_val)

#     # Calculate the intersection and union of the ranges
#     if isinstance(range1, list) or isinstance(range2, list):
#         # Handle != case where range might be a list of two ranges
#         intersection = []
#         union = [(min_val, max_val)]
#         if isinstance(range1, list):
#             for r in range1:
#                 if isinstance(range2, list):
#                     for s in range2:
#                         intersection.append(max(r[0], s[0]))
#                         intersection.append(min(r[1], s[1]))
#                 else:
#                     intersection.append(max(r[0], range2[0]))
#                     intersection.append(min(r[1], range2[1]))
#         else:
#             for s in range2:
#                 intersection.append(max(range1[0], s[0]))
#                 intersection.append(min(range1[1], s[1]))
#     else:
#         intersection = (max(range1[0], range2[0]), min(range1[1], range2[1]))
#         union = (min(range1[0], range2[0]), max(range1[1], range2[1]))

#     # Calculate intersection length
#     if intersection[0] <= intersection[1]:
#         intersection_length = intersection[1] - intersection[0]
#     else:
#         intersection_length = 0

#     # Calculate union length
#     union_length = union[1] - union[0]

#     # Calculate the Jaccard similarity
#     similarity = intersection_length / union_length
#     return similarity



# variable_ranges = load_variable_ranges_from_file("variables_config.json")

# expr1 = "height > 1"
# expr2 = "height > 1"

# similarity = calculate_jaccard_similarity(expr1, expr2, variable_ranges)
# similarity

In [49]:
#VERSAO 2
##############

# import re
# import json

# def parse_expression(expression):
#     """
#     Parse the expression into its components: variable, operator, and value.
#     """
#     match = re.match(r"(\w+)\s*(>=|<=|>|<|==|!=)\s*(.+)", expression)
#     if match:
#         variable, operator, value = match.groups()
#         return variable, operator, value
#     else:
#         raise ValueError(f"Invalid expression format: {expression}")

# def load_variable_ranges_from_file(file_path):
#     with open(file_path, 'r') as file:
#         json_data = file.read()
#     variable_ranges = json.loads(json_data)
#     return {var['name']: var for var in variable_ranges}

# def get_expression_range(var_name, operator, value, min_val, max_val):
#     """
#     Determine the range of values that satisfy the expression.
#     """
#     value = float(value)
#     if operator == '>=':
#         return (value, max_val)
#     elif operator == '<=':
#         return (min_val, value)
#     elif operator == '>':
#         return (value + 1e-9, max_val)  # Adding a small value to make it exclusive
#     elif operator == '<':
#         return (min_val, value - 1e-9)  # Subtracting a small value to make it exclusive
#     elif operator == '==':
#         return (value, value)
#     elif operator == '!=':
#         return [(min_val, value - 1e-9), (value + 1e-9, max_val)]  # Returns two ranges
#     else:
#         raise ValueError(f"Unsupported operator: {operator}")

# def calculate_jaccard_similarity(expr1, expr2, variable_ranges):

#     """
#     Calculate the Jaccard similarity between two expressions.
#     The expressions should be of the format: "variable operator value".
#     """
#     try:
#         var1, op1, val1 = parse_expression(expr1)
#         var2, op2, val2 = parse_expression(expr2)
#     except ValueError as e:
#         print(e)
#         return 0.0

#     # Ensure the variable names are the same
#     if var1 != var2:
#         return 0.0

#     # Get variable ranges
#     var_info = variable_ranges.get(var1)
#     if not var_info:
#         print(f"Variable {var1} not found in ranges.")
#         return 0.0

#     min_val = var_info['min_value']
#     max_val = var_info['max_value']

#     # Get ranges for each expression
#     range1 = get_expression_range(var1, op1, val1, min_val, max_val)
#     range2 = get_expression_range(var2, op2, val2, min_val, max_val)

#     # Calculate intersection and union considering the equality operator cases
#     if op1 == '==' and op2 != '==':
#         if range2[0] <= range1[0] <= range2[1]:
#             intersection_length = 1
#             union_length = range2[1] - range2[0]
#         else:
#             return 0.0
#     elif op2 == '==' and op1 != '==':
#         if range1[0] <= range2[0] <= range1[1]:
#             intersection_length = 1
#             union_length = range1[1] - range1[0]
#         else:
#             return 0.0
#     elif op1 == '==' and op2 == '==':
#         if val1 == val2:
#             return 1.0
#         else:
#             return 0.0
#     else:
#         if isinstance(range1, list) or isinstance(range2, list):
#             # Handle != case where range might be a list of two ranges
#             intersection = []
#             union = [(min_val, max_val)]
#             if isinstance(range1, list):
#                 for r in range1:
#                     if isinstance(range2, list):
#                         for s in range2:
#                             intersection.append((max(r[0], s[0]), min(r[1], s[1])))
#                     else:
#                         intersection.append((max(r[0], range2[0]), min(r[1], range2[1])))
#             else:
#                 for s in range2:
#                     intersection.append((max(range1[0], s[0]), min(range1[1], s[1])))
#         else:
#             intersection = (max(range1[0], range2[0]), min(range1[1], range2[1]))
#             union = (min(range1[0], range2[0]), max(range1[1], range2[1]))

#         # Calculate intersection length
#         if isinstance(intersection, list):
#             intersection_length = 0
#             for r in intersection:
#                 if r[0] <= r[1]:
#                     intersection_length += r[1] - r[0]
#         else:
#             if intersection[0] <= intersection[1]:
#                 intersection_length = intersection[1] - intersection[0]
#             else:
#                 intersection_length = 0

#         # Calculate union length
#         union_length = union[1] - union[0] if isinstance(union, tuple) else max_val - min_val

#     # Calculate the Jaccard similarity
#     similarity = intersection_length / union_length
#     return similarity

# # Load the variable ranges from the file
# variable_ranges = load_variable_ranges_from_file('variables_config.json')

# # Test cases
# test_cases = [
#     ("height >= 30", "height == 40"),  # equality with range
#     ("height == 30", "height == 30"),  # equality with equality (same)
#     ("height == 30", "height == 40"),  # equality with equality (different)
#     ("height >= 30", "height <= 40"),  # range with range
#     ("height != 30", "height <= 40"),  # inequality with range
# ]

# results = {f"{leaf1} vs {leaf2}": calculate_jaccard_similarity(leaf1, leaf2, variable_ranges)
#            for leaf1, leaf2 in test_cases}

# results


In [50]:
def parse_expression(leaf):
    """
    Parse the expression into its components: variable, operator, and value.
    """
    match = re.match(r"(\w+)\s*(>=|<=|>|<|==|!=)\s*(.+)", leaf)
    if match:
        variable, operator, value = match.groups()
        return variable, operator, value
    else:
        raise ValueError(f"Invalid expression format: {leaf}")

def get_expression_range(var_name, operator, value, min_val, max_val):
    """
    Determine the range of values that satisfy the expression.
    """
    value = float(value)
    if operator == '>=':
        return (value, max_val)
    elif operator == '<=':
        return (min_val, value)
    elif operator == '>':
        return (value + 1e-9, max_val)  # Adding a small value to make it exclusive
    elif operator == '<':
        return (min_val, value - 1e-9)  # Subtracting a small value to make it exclusive
    elif operator == '==':
        return (value, value)
    elif operator == '!=':
        return [(min_val, value - 1e-9), (value + 1e-9, max_val)]  # Returns two ranges
    else:
        raise ValueError(f"Unsupported operator: {operator}")

def calculate_jaccard_similarity(leaf1, leaf2, variable_ranges):
    """
    Calculate the Jaccard similarity between two expressions.
    The expressions should be of the format: "variable operator value".
    """
    try:
        var1, op1, val1 = parse_expression(leaf1)
        var2, op2, val2 = parse_expression(leaf2)
    except ValueError as e:
        print(e)
        return 0.0

    # Ensure the variable names are the same
    if var1 != var2:
        return 0.0

    # Get variable ranges
    var_info = variable_ranges.get(var1)
    if not var_info:
        print(f"Variable {var1} not found in ranges.")
        return 0.0

    min_val = var_info['min_value']
    max_val = var_info['max_value']

    # Get ranges for each expression
    range1 = get_expression_range(var1, op1, val1, min_val, max_val)
    range2 = get_expression_range(var2, op2, val2, min_val, max_val)

    def calculate_intersection_union(range1, range2):
        """
        Helper function to calculate intersection and union of two ranges.
        """
        if isinstance(range1, list):
            intersections = []
            for r1 in range1:
                if isinstance(range2, list):
                    for r2 in range2:
                        intersections.append((max(r1[0], r2[0]), min(r1[1], r2[1])))
                else:
                    intersections.append((max(r1[0], range2[0]), min(r1[1], range2[1])))
        else:
            if isinstance(range2, list):
                intersections = [(max(range1[0], r2[0]), min(range1[1], r2[1])) for r2 in range2]
            else:
                intersections = [(max(range1[0], range2[0]), min(range1[1], range2[1]))]
        
        valid_intersections = [r for r in intersections if r[0] <= r[1]]
        intersection_length = sum(r[1] - r[0] for r in valid_intersections)
        
        union_min = min(range1[0] if not isinstance(range1, list) else min(r[0] for r in range1),
                        range2[0] if not isinstance(range2, list) else min(r[0] for r in range2))
        union_max = max(range1[1] if not isinstance(range1, list) else max(r[1] for r in range1),
                        range2[1] if not isinstance(range2, list) else max(r[1] for r in range2))
        union_length = union_max - union_min
        
        return intersection_length, union_length

    # Calculate intersection and union considering the equality operator cases
    if op1 == '==' and op2 != '==':
        if isinstance(range2, list):
            range2_min, range2_max = min(range2[0][0], range2[1][0]), max(range2[0][1], range2[1][1])
            if range2_min <= range1[0] <= range2_max:
                intersection_length = 1
                union_length = range2_max - range2_min
            else:
                return 0.0
        else:
            if range2[0] <= range1[0] <= range2[1]:
                intersection_length = 1
                union_length = range2[1] - range2[0]
            else:
                return 0.0
    elif op2 == '==' and op1 != '==':
        if isinstance(range1, list):
            range1_min, range1_max = min(range1[0][0], range1[1][0]), max(range1[0][1], range1[1][1])
            if range1_min <= range2[0] <= range1_max:
                intersection_length = 1
                union_length = range1_max - range1_min
            else:
                return 0.0
        else:
            if range1[0] <= range2[0] <= range1[1]:
                intersection_length = 1
                union_length = range1[1] - range1[0]
            else:
                return 0.0
    elif op1 == '==' and op2 == '==':
        if val1 == val2:
            return 1.0
        else:
            return 0.0
    else:
        intersection_length, union_length = calculate_intersection_union(range1, range2)

    # Calculate the Jaccard similarity
    similarity = intersection_length / union_length
    return similarity

# Load the variable ranges from the file
# variable_ranges_from_file = load_variable_ranges()

# Test cases
# test_cases = [
#     ("height == 30", "height <= 40"),  # equality with range
#     ("height == 30", "height == 30"),  # equality with equality (same)
#     ("height == 30", "height == 40"),  # equality with equality (different)
#     ("height >= 30", "height <= 40"),  # range with range
#     ("height != 30", "height <= 40"),  # inequality with range
#     ("height != 30", "height == 40")   # inequality with equality
# ]

# var_similarity_dict = {f"{expr1} vs {expr2}": calculate_jaccard_similarity(expr1, expr2, variable_ranges_from_file)
#            for expr1, expr2 in test_cases}

# # # Convert results to DataFrame and display
# # results_df = pd.DataFrame(results.items(), columns=["Test Case", "Similarity"])
# # results_df
# var_similarity_dict




In [51]:
def local_similarity(leaf1, leaf2, variable_ranges_from_file):
    local_similarity = calculate_jaccard_similarity(leaf1, leaf2, variable_ranges_from_file)
    # print(f"Local similarity between {leaf1} vs {leaf2}: {local_similarity}")
    return local_similarity

In [52]:
def get_leafs(dnf):
    parsed_expression = logical_expression_settings().parseString(dnf, parseAll=True)
    # print("Parsed Expression:", parsed_expression)

    extracted_leaves=extract_leaves(parsed_expression.asList())
    # print("Extracted Leafs:", extracted_leaves)
    return extracted_leaves

In [53]:
def extract_variables(leafts):
    """
    Extracts the variable names from a list of leaf strings.

    Parameters:
    leaves (list): List of strings, each containing a variable, an operator, and a value.

    Returns:
    list: List of variable names.
    """
    variables = []
    for leaf in leafts:
        # Using regular expression to match the variable name at the beginning of the string
        match = re.match(r"([a-zA-Z_]\w*)\s*[<>]=?|==|!=\s*\d+", leaf)
        if match:
            variables.append(match.group(1))
    return variables

In [54]:
def tversky_similarity(vars1, vars2, alpha=1.0, beta=1.0):
    """
    Calculate the Tversky similarity between two sets of variables.
    
    Args:
    vars1 (list): First list of variables.
    vars2 (list): Second list of variables.
    alpha (float): Weight for the elements unique to vars1. Default is 1.0.
    beta (float): Weight for the elements unique to vars2. Default is 1.0.
    
    Returns:
    float: The Tversky similarity score.
    """
    set1 = set(vars1)
    set2 = set(vars2)
    
    intersection = len(set1 & set2)
    only_in_set1 = len(set1 - set2)
    only_in_set2 = len(set2 - set1)
    
    tversky_index = intersection / (intersection + alpha * only_in_set1 + beta * only_in_set2)
    
    return tversky_index

def penality(vars1, vars2, alpha=1.0, beta=1.0):
    return 1 - tversky_similarity(vars1, vars2, alpha, beta)


In [55]:
def intersection(vector1, vector2):
    """
    Find the intersection between two vectors of elements.
    
    Args:
    vector1 (list): First list of elements.
    vector2 (list): Second list of elements.
    
    Returns:
    list: A list containing the elements that are present in both vectors.
    """
    set1 = set(vector1)
    set2 = set(vector2)
    
    intersection = set1 & set2
    
    return list(intersection)


In [56]:
def pair_leaves(leaves1, leaves2):
    """
    Create pairs of leaves containing the same variable.
    
    Args:
    leaves1 (list): First list of leaves.
    leaves2 (list): Second list of leaves.
    
    Returns:
    list: A list of tuples, each containing a pair of leaves with the same variable.
    """
    pairs = []
    
    # Extract variable names from leaves
    def extract_variable(leaf):
        return leaf.split()[0]
    
    variables1 = {extract_variable(leaf): leaf for leaf in leaves1}
    variables2 = {extract_variable(leaf): leaf for leaf in leaves2}
    
    # Create pairs
    for var in variables1:
        if var in variables2:
            pairs.append((variables1[var], variables2[var]))
    
    return pairs


In [57]:
# funcao Mocked para similar os pesos.
def get_weights():
    # TODO: Implementar a lógica para obter os pesos com esse formato {'a': 0.75, 'b': 0.0, 'c': 0.0}
    return {} # Weight is considered 1 when variable is not present in the weights dictionary

weight_dict = get_weights()
# print("Weights:{}".format(weight_dict))

In [58]:
def extract_variable_similarity(local_similarity):
    """
    Extract variable names and their similarities from a dictionary of leaf pairs.
    
    Args:
    pairs_similarity (dict): A dictionary with tuples of leaf pairs as keys and similarity values as values.
    
    Returns:
    dict: A dictionary with variable names as keys and similarity values as values.
    """
    variable_similarity = {}
    
    for (leaf1, leaf2), similarity in local_similarity.items():
        variable = leaf1.split()[0]
        variable_similarity[variable] = similarity
    
    return variable_similarity

In [59]:
def similarity_global(local_similarity):
    weight_dict=get_weights()
    variable_similarity_dict=extract_variable_similarity(local_similarity)

    weighted_sum = 0
    total_weight = 0
    
    for var, similarity in variable_similarity_dict.items():
        weight = weight_dict.get(var, 1)
        weighted_sum += similarity * weight
        total_weight += weight

    if total_weight == 0:
        return 0  # Avoid division by zero
    
    weighted_average = weighted_sum / total_weight
    return weighted_average

In [60]:
desired_scenario=get_desired_scenario()

given_dnf1=to_dnf_expression(get_Given(desired_scenario))
when_dnf1=to_dnf_expression(get_When(desired_scenario))
then_dnf1=to_dnf_expression(get_Then(desired_scenario))

print("Desired Given DNF: ", given_dnf1)
print("Desired When DNF: ", when_dnf1)
print("Desired Then DNF: ", then_dnf1)
print("")

given_leafs1=get_leafs(given_dnf1)
when_leafs1=get_leafs(when_dnf1)
then_leafs1=get_leafs(then_dnf1)

print("Leafs Given 1: ", given_leafs1)
print("Leafs When 1: ", when_leafs1)
print("Leafs Then 1: ", then_leafs1)

given_variables1=extract_variables(given_leafs1)
when_variables1=extract_variables(when_leafs1)
then_variables1=extract_variables(then_leafs1)

print("Variables Given 1: ", given_variables1)
print("Variables When 1: ", when_variables1)
print("Variables Then 1: ", then_variables1)

print("--")

shared_scenarios = get_shared_scenarios()

similarity_result = {} 
for shared_scenario in shared_scenarios:
    given_dnf2=to_dnf_expression(get_Given(shared_scenario))
    when_dnf2=to_dnf_expression(get_When(shared_scenario))
    then_dnf2=to_dnf_expression(get_Then(shared_scenario))

    print("Shared Given DNF: ", given_dnf2)
    print("Shared When DNF: ", when_dnf2)
    print("Shared Then DNF: ", then_dnf2)
    print("")

    given_leafs2=get_leafs(given_dnf2)
    when_leafs2=get_leafs(when_dnf2)
    then_leafs2=get_leafs(then_dnf2)

    print("Leafs Given 2: ", given_leafs2)
    print("Leafs When 2: ", when_leafs2)
    print("Leafs Then 2: ", then_leafs2)
    print("")
    given_variables2=extract_variables(given_leafs2)
    when_variables2=extract_variables(when_leafs2)
    then_variables2=extract_variables(then_leafs2)

    print("Variables Given 2: ", given_variables2)
    print("Variables When 2: ", when_variables2)
    print("Variables Then 2: ", then_variables2)
    print("")

    penality_given=penality(given_variables1, given_variables2)
    penality_when=penality(when_variables1, when_variables2)
    penality_then=penality(then_variables1, then_variables2)

    print("Penality Given: ", penality_given)
    print("Penality When: ", penality_when)
    print("Penality Then: ", penality_then)
    print("")

    pair_leaves_given=pair_leaves(given_leafs1, given_leafs2)
    pair_leaves_when=pair_leaves(when_leafs1, when_leafs2)
    pair_leaves_then=pair_leaves(then_leafs1, then_leafs2)

    print("Pair Leaves Given: ", pair_leaves_given)
    print("Pair Leaves When: ", pair_leaves_when)
    print("Pair Leaves Then: ", pair_leaves_then)
    print("")

    given_local_similarity_result={}
    when_local_similarity_result={}
    then_local_similarity_result={}

    for pair in pair_leaves_given:
        given_local_similarity_result[pair]=local_similarity(pair[0], pair[1], load_variable_ranges())

    for pair in pair_leaves_when:
        when_local_similarity_result[pair]=local_similarity(pair[0], pair[1], load_variable_ranges())

    for pair in pair_leaves_then:
        then_local_similarity_result[pair]=local_similarity(pair[0], pair[1], load_variable_ranges())


    print("Given Local Similarity Result: ", given_local_similarity_result)
    print("When Local Similarity Result: ", when_local_similarity_result)
    print("Then Local Similarity Result: ", then_local_similarity_result)
    print("")

    global_similarity_given=similarity_global(given_local_similarity_result)
    global_similarity_when=similarity_global(when_local_similarity_result)
    global_similarity_then=similarity_global(then_local_similarity_result)

    print("Global Similarity Given: ", global_similarity_given)
    print("Global Similarity When: ", global_similarity_when)
    print("Global Similarity Then: ", global_similarity_then)
    print("")
    
final_similarity_given=global_similarity_given-penality_given
final_similarity_when=global_similarity_when-penality_when
final_similarity_then=global_similarity_then-penality_then

print("Final Similarity Given: ", final_similarity_given)
print("Final Similarity When: ", final_similarity_when)
print("Final Similarity Then: ", final_similarity_then)
print("")

Desired Given DNF:  height > 3 AND obstacle > 10
Desired When DNF:  distance_target <= 1
Desired Then DNF:  height < 10 AND distance_target <= 5

Leafs Given 1:  ['height > 3', 'obstacle > 10']
Leafs When 1:  ['distance_target <= 1']
Leafs Then 1:  ['height < 10', 'distance_target <= 5']
Variables Given 1:  ['height', 'obstacle']
Variables When 1:  ['distance_target']
Variables Then 1:  ['height', 'distance_target']
--
Shared Given DNF:  height > 3 AND obstacle < 10
Shared When DNF:  distance_target <= 5
Shared Then DNF:  height < 10 AND distance_target <= 5

Leafs Given 2:  ['height > 3', 'obstacle < 10']
Leafs When 2:  ['distance_target <= 5']
Leafs Then 2:  ['height < 10', 'distance_target <= 5']

Variables Given 2:  ['height', 'obstacle']
Variables When 2:  ['distance_target']
Variables Then 2:  ['height', 'distance_target']

Penality Given:  0.0
Penality When:  0.0
Penality Then:  0.0

Pair Leaves Given:  [('height > 3', 'height > 3'), ('obstacle > 10', 'obstacle < 10')]
Pair Leav